# 5반 16조: 기가지니 연계 1인가구 서비스 기술실증

In [2]:
%%writefile bigproject_testpage.py

# 라이브러리 불러오기
import streamlit as st
from streamlit_folium import st_folium
import pandas as pd
from keras.models import load_model
import os, glob
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
import matplotlib.pyplot as plt
import plotly.express as px

os.chdir('C:/Users/User/Desktop/빅프/프로토타입/big_project_230613_오전/Data')

########################### ARIMA 모델 함수 ###########################################
def Lone_Person_Dataset_Loader(group_name, region_name, gender_name, age_name):
    
    df = pd.read_csv('./Lone_Person_Data/group_n.csv')

    # 20, 25=>20대 ~ 70, 75=>70대 연령대 전처리
    df['연령대'] = df['연령대'].astype(float)
    conditions = [
        (df['연령대'] >= 20) & (df['연령대'] < 30),
        (df['연령대'] >= 30) & (df['연령대'] < 40),
        (df['연령대'] >= 40) & (df['연령대'] < 50),
        (df['연령대'] >= 50) & (df['연령대'] < 60),
        (df['연령대'] >= 60) & (df['연령대'] < 70),
        (df['연령대'] >= 70) & (df['연령대'] < 80)
    ]
    values = ['20대', '30대', '40대', '50대', '60대', '70대']

    # 성별, 날짜 전처리
    df['연령대'] = np.select(conditions, values, default='80대')
    df['성별'] = df['성별'].replace({1: '남성', 2: '여성'})
    df['month'] = df['month'].apply(lambda x: str('-'.join(str(x).split('.'))))
    df['month'] = pd.to_datetime(df['month'])
    df['month'] = df['month'].dt.to_period('M')
    x_train = df.loc[(df['month'] != '2023-01') & (df['month'] != '2023-02') & (df['month'] != '2023-03')]
    x_test = df.loc[(df['month'] == '2023-01') | (df['month'] == '2023-02') | (df['month'] == '2023-03')]
    
    df_ts = x_train.groupby(['month', '자치구', '성별', '연령대']).sum().reset_index()

    # 시계열 분석을 위한 데이터 프레임 재구성(피봇테이블)
    df_pivot = df_ts.pivot_table(index='month', columns=['자치구', '성별', '연령대'], values=group_name)
    frame = pd.DataFrame(df_pivot[region_name, gender_name, age_name])
    df_col = df_pivot[region_name, gender_name, age_name].fillna(0)
    column_name = frame.columns.values[0]
    
    # PeriodIndex를 DatetimeIndex으로 바꾸기 ('M'한달 간격)
    new_index = df_col.index.to_timestamp(freq='M')

    # Datetimeindex 바꾼 데이터프레임
    df_col = pd.DataFrame(df_pivot[region_name, gender_name, age_name].values, index=new_index, columns=[column_name])

    # ARIMA 모델 학습
    model = ARIMA(df_col, order=(2, 1, 1))  # p, d, q
    fitted_model = model.fit()

    # 2023 예측
    start_idx = len(df_col) - 1  # 지난 관측값에서 시작
    end_idx = start_idx + 12  # 12개월치 예측
    forecast = fitted_model.predict(start=start_idx, end=end_idx, typ='levels')
    forecast_index = pd.date_range(start=df_col.index[-1], periods=13, freq='M')[1:]
    forecast = pd.Series(forecast, index=forecast_index)
   
    df_ts2 = x_test.groupby(['month', '자치구', '성별', '연령대']).sum().reset_index()

    # 시계열 분석을 위한 데이터 프레임 재구성(피봇테이블) 
    df_pivot2 = df_ts2.pivot_table(index='month', columns=['자치구', '성별', '연령대'], values=group_name)
    
    # 한글 설정
    import matplotlib.font_manager as fm
    # 폰트 경로 설정
    plt.rc('font', family='Malgun Gothic')
    df_col2 = df_pivot2[region_name, gender_name, age_name].fillna(0)
    from sklearn.metrics import mean_squared_error

    # 예측값과 실제값 사이의 MSE 계산
    mse = mean_squared_error(df_pivot2[region_name, gender_name, age_name], forecast[0:3], squared=False)
    
    # 예측 결과 시계열차트
    plt.figure(figsize=(10, 6))
    fig, ax = plt.subplots(figsize=(10, 6))

    ax.plot(df_col.index.strftime('%Y-%m-%d'), df_pivot[region_name, gender_name, age_name], marker='o', color='pink', label='실제값')
    for i in range(len(df_col.index)):
        height = df_pivot[region_name, gender_name, age_name][i]
        ax.text(df_col.index[i].strftime('%Y-%m-%d'), height + 0.25, '%.1f' % height, ha='center', va='bottom', size=10)

    ax.plot(forecast.index.strftime('%Y-%m-%d'), forecast, label='예측값', marker='o', color='gray')
    for i in range(len(forecast.index)):
        height = forecast[i]
        ax.text(forecast.index[i].strftime('%Y-%m-%d'), height + 1, '%.1f' % height, ha='center', va='bottom', size=10, color='gray')

    # ax.plot(df_col2.index.strftime('%Y-%m-%d'), df_pivot2[region_name, gender_name, age_name], label='미래 실제값', marker='o', color='pink')
    # for i in range(len(df_col2.index)):
    #     height = df_pivot2[region_name, gender_name, age_name][i]
    #     ax.text(df_col2.index[i].strftime('%Y-%m-%d'), height + 0.25, '%.1f' % height, ha='center', va='bottom', size=10)

    ax.set_title(str(column_name) + ' ' + group_name)
    plt.xticks(rotation=45)
    ax.set_xlabel('날짜')
    ax.set_ylabel('1인 가구 수')
    ax.legend()

    plt.tight_layout()
    st.pyplot(fig)

    
########################### 예측값 출력 함수 ###########################################
def pred(group_name, region_name, gender_name, age_name):
    df = pd.read_csv('./Lone_Person_Data/group_n.csv')

    # 20, 25=>20대 ~ 70, 75=>70대 연령대 전처리
    df['연령대'] = df['연령대'].astype(float)
    conditions = [
        (df['연령대'] >= 20) & (df['연령대'] < 30),
        (df['연령대'] >= 30) & (df['연령대'] < 40),
        (df['연령대'] >= 40) & (df['연령대'] < 50),
        (df['연령대'] >= 50) & (df['연령대'] < 60),
        (df['연령대'] >= 60) & (df['연령대'] < 70),
        (df['연령대'] >= 70) & (df['연령대'] < 80)
    ]
    values = ['20대', '30대', '40대', '50대', '60대', '70대']

    # 성별, 날짜 전처리
    df['연령대'] = np.select(conditions, values, default='80대')
    df['성별'] = df['성별'].replace({1: '남성', 2: '여성'})
    df['month'] = df['month'].apply(lambda x: str('-'.join(str(x).split('.'))))
    df['month'] = pd.to_datetime(df['month'])
    df['month'] = df['month'].dt.to_period('M')
    x_train = df.loc[(df['month'] != '2023-01') & (df['month'] != '2023-02') & (df['month'] != '2023-03')]
    x_test = df.loc[(df['month'] == '2023-01') | (df['month'] == '2023-02') | (df['month'] == '2023-03')]
    
    df_ts = x_train.groupby(['month', '자치구', '성별', '연령대']).sum().reset_index()

    # 시계열 분석을 위한 데이터 프레임 재구성(피봇테이블)
    df_pivot = df_ts.pivot_table(index='month', columns=['자치구', '성별', '연령대'], values=group_name)
    frame = pd.DataFrame(df_pivot[region_name, gender_name, age_name])
    df_col = df_pivot[region_name, gender_name, age_name].fillna(0)
    column_name = frame.columns.values[0]
    
    # PeriodIndex를 DatetimeIndex으로 바꾸기 ('M'한달 간격)
    new_index = df_col.index.to_timestamp(freq='M')

    # Datetimeindex 바꾼 데이터프레임
    df_col = pd.DataFrame(df_pivot[region_name, gender_name, age_name].values, index=new_index, columns=[column_name])

    # ARIMA 모델 학습
    model = ARIMA(df_col, order=(2, 1, 1))  # p, d, q
    fitted_model = model.fit()

    # 2023 예측
    start_idx = len(df_col) - 1  # 지난 관측값에서 시작
    end_idx = start_idx + 12  # 12개월치 예측
    forecast = fitted_model.predict(start=start_idx, end=end_idx, typ='levels')
    forecast_index = pd.date_range(start=df_col.index[-1], periods=13, freq='M')[1:]
    forecast = pd.Series(forecast, index=forecast_index)
   
    df_ts2 = x_test.groupby(['month', '자치구', '성별', '연령대']).sum().reset_index()

    # 시계열 분석을 위한 데이터 프레임 재구성(피봇테이블)
    df_pivot2 = df_ts2.pivot_table(index='month', columns=['자치구', '성별', '연령대'], values=group_name)
    
    # 한글 설정
    import matplotlib.font_manager as fm
    # 폰트 경로 설정
    plt.rc('font', family='Malgun Gothic')
    df_col2 = df_pivot2[region_name, gender_name, age_name].fillna(0)
    
    st.write(forecast)



    
########################### 파이차트(현재) 함수 ###########################################
def piechart(region, gender, age):
    df = pd.read_csv('./Lone_Person_Data/group_n.csv')

    # 20, 25=>20대 ~ 70, 75=>70대 연령대 전처리
    df['연령대'] = df['연령대'].astype(float)
    conditions = [
        (df['연령대'] >= 20) & (df['연령대'] < 30),
        (df['연령대'] >= 30) & (df['연령대'] < 40),
        (df['연령대'] >= 40) & (df['연령대'] < 50),
        (df['연령대'] >= 50) & (df['연령대'] < 60),
        (df['연령대'] >= 60) & (df['연령대'] < 70),
        (df['연령대'] >= 70) & (df['연령대'] < 80)
    ]
    values = ['20대', '30대', '40대', '50대', '60대', '70대']

    # 성별, 날짜 전처리
    df['연령대'] = np.select(conditions, values, default='80대')
    df['성별'] = df['성별'].replace({1: '남성', 2: '여성'})
    df['month'] = df['month'].apply(lambda x: str('-'.join(str(x).split('.'))))
    df['month'] = pd.to_datetime(df['month'])
    df['month'] = df['month'].dt.to_period('M')
    x_train = df.loc[(df['month'] != '2023-01') & (df['month'] != '2023-02') & (df['month'] != '2023-03')]
    x_test = df.loc[(df['month'] == '2023-01') | (df['month'] == '2023-02') | (df['month'] == '2023-03')]

    df_ts = x_train.groupby(['month', '자치구', '성별', '연령대']).sum().reset_index()
    
    # 시계열 분석을 위한 데이터 프레임 재구성(피봇테이블) 자치구, 성별, 연령대별 각 집단 값의 피봇테이블
    df_pivot3_1 = df_ts.pivot_table(index='month', columns=['자치구', '성별', '연령대'], values='커뮤니케이션이 적은 집단')
    df_pivot3_2 = df_ts.pivot_table(index='month', columns=['자치구', '성별', '연령대'], values='평일 외출이 적은 집단')
    df_pivot3_3 = df_ts.pivot_table(index='month', columns=['자치구', '성별', '연령대'], values='휴일 외출이 적은 집단')
    df_pivot3_4 = df_ts.pivot_table(index='month', columns=['자치구', '성별', '연령대'], values='출근소요시간 및 근무시간이 많은 집단')
    df_pivot3_5 = df_ts.pivot_table(index='month', columns=['자치구', '성별', '연령대'], values='외출이 매우 적은 집단(전체)')
    df_pivot3_6 = df_ts.pivot_table(index='month', columns=['자치구', '성별', '연령대'], values='외출이 매우 많은 집단')
    df_pivot3_7 = df_ts.pivot_table(index='month', columns=['자치구', '성별', '연령대'], values='동영상서비스 이용이 많은 집단')
    df_pivot3_8 = df_ts.pivot_table(index='month', columns=['자치구', '성별', '연령대'], values='생활서비스 이용이 많은 집단')
    df_pivot3_9 = df_ts.pivot_table(index='month', columns=['자치구', '성별', '연령대'], values='재정상태에 대한 관심집단')
    df_pivot3_10 = df_ts.pivot_table(index='month', columns=['자치구', '성별', '연령대'], values='외출-커뮤니케이션이 모두 적은 집단(전체)')
    
    value1 = df_pivot3_1[region, gender, age]['2022-12']
    value2 = df_pivot3_2[region, gender, age]['2022-12']
    value3 = df_pivot3_3[region, gender, age]['2022-12']
    value4 = df_pivot3_4[region, gender, age]['2022-12']
    value5 = df_pivot3_5[region, gender, age]['2022-12']
    value6 = df_pivot3_6[region, gender, age]['2022-12']
    value7 = df_pivot3_7[region, gender, age]['2022-12']
    value8 = df_pivot3_8[region, gender, age]['2022-12']
    value9 = df_pivot3_9[region, gender, age]['2022-12']
    value10 = df_pivot3_10[region, gender, age]['2022-12']
    total = value1+value2+value3+value4+value5+value6+value7+value8+value9+value10

    # 파이차트: 자치구, 성별, 연령을 지정하면 현재(2022-12), 미래(2023-12) 1인 가구 집단의 비중
    ratio = [value1/total, value2/total, value3/total, value4/total, value5/total, value6/total, value7/total, value8/total, value9/total, value10/total]
    labels = ['커뮤니케이션이 적은 집단',
           '평일 외출이 적은 집단', '휴일 외출이 적은 집단', '출근소요시간 및 근무시간이 많은 집단',
           '외출이 매우 적은 집단(전체)', '외출이 매우 많은 집단', '동영상서비스 이용이 많은 집단',
           '생활서비스 이용이 많은 집단', '재정상태에 대한 관심집단', '외출-커뮤니케이션이 모두 적은 집단(전체)']
    
    fig = px.pie(values=ratio, names=labels, hole=0.3)
    fig.update_traces(textposition='inside', textinfo='percent+label')
    fig.update_layout(font=dict(size=16))
    return fig



########################### 파이차트(미래) 함수 ###########################################
def piechart_pred(region, gender, age):
    df = pd.read_csv('./Lone_Person_Data/group_n.csv')

    # 20, 25=>20대 ~ 70, 75=>70대 연령대 전처리
    df['연령대'] = df['연령대'].astype(float)
    conditions = [
        (df['연령대'] >= 20) & (df['연령대'] < 30),
        (df['연령대'] >= 30) & (df['연령대'] < 40),
        (df['연령대'] >= 40) & (df['연령대'] < 50),
        (df['연령대'] >= 50) & (df['연령대'] < 60),
        (df['연령대'] >= 60) & (df['연령대'] < 70),
        (df['연령대'] >= 70) & (df['연령대'] < 80)
    ]
    values = ['20대', '30대', '40대', '50대', '60대', '70대']

    # 성별, 날짜 전처리
    df['연령대'] = np.select(conditions, values, default='80대')
    df['성별'] = df['성별'].replace({1: '남성', 2: '여성'})
    df['month'] = df['month'].apply(lambda x: str('-'.join(str(x).split('.'))))
    df['month'] = pd.to_datetime(df['month'])
    df['month'] = df['month'].dt.to_period('M')
    x_train = df.loc[(df['month'] != '2023-01') & (df['month'] != '2023-02') & (df['month'] != '2023-03')]
    x_test = df.loc[(df['month'] == '2023-01') | (df['month'] == '2023-02') | (df['month'] == '2023-03')]

    df_ts = x_train.groupby(['month', '자치구', '성별', '연령대']).sum().reset_index()
    
    # 시계열 분석을 위한 데이터 프레임 재구성(피봇테이블) 자치구, 성별, 연령대별 각 집단 값의 피봇테이블
    df_pivot3_1 = df_ts.pivot_table(index='month', columns=['자치구', '성별', '연령대'], values='커뮤니케이션이 적은 집단')
    df_pivot3_2 = df_ts.pivot_table(index='month', columns=['자치구', '성별', '연령대'], values='평일 외출이 적은 집단')
    df_pivot3_3 = df_ts.pivot_table(index='month', columns=['자치구', '성별', '연령대'], values='휴일 외출이 적은 집단')
    df_pivot3_4 = df_ts.pivot_table(index='month', columns=['자치구', '성별', '연령대'], values='출근소요시간 및 근무시간이 많은 집단')
    df_pivot3_5 = df_ts.pivot_table(index='month', columns=['자치구', '성별', '연령대'], values='외출이 매우 적은 집단(전체)')
    df_pivot3_6 = df_ts.pivot_table(index='month', columns=['자치구', '성별', '연령대'], values='외출이 매우 많은 집단')
    df_pivot3_7 = df_ts.pivot_table(index='month', columns=['자치구', '성별', '연령대'], values='동영상서비스 이용이 많은 집단')
    df_pivot3_8 = df_ts.pivot_table(index='month', columns=['자치구', '성별', '연령대'], values='생활서비스 이용이 많은 집단')
    df_pivot3_9 = df_ts.pivot_table(index='month', columns=['자치구', '성별', '연령대'], values='재정상태에 대한 관심집단')
    df_pivot3_10 = df_ts.pivot_table(index='month', columns=['자치구', '성별', '연령대'], values='외출-커뮤니케이션이 모두 적은 집단(전체)')
    
    # 10개 집단에 대한 예측치 forecast1~10개 추출
    forecast_list = []
    for i in range(1, 11):
        df_col = locals()[f'df_pivot3_{i}'][region, gender, age].fillna(0)

        # PeriodIndex를 DatetimeIndex으로 바꾸기 ('M'한달 간격)
        new_index = df_col.index.to_timestamp(freq='M')
        frame = pd.DataFrame(locals()[f'df_pivot3_{i}'][region, gender, age])
        column_name = frame.columns.values[0]

        # Datetimeindex 바꾼 데이터프레임
        df_col = pd.DataFrame(locals()[f'df_pivot3_{i}'][region, gender, age].values, index=new_index, columns=[column_name])

        # 모델 학습
        model = ARIMA(df_col, order=(2, 1, 1))  # p, d, q
        fitted_model = model.fit()

        # 2023 예측
        start_idx = len(df_col) - 1  # 지난 관측값에서 시작
        end_idx = start_idx + 12  # 12개월치 예측
        forecast = fitted_model.predict(start=start_idx, end=end_idx, typ='levels')
        forecast_index = pd.date_range(start=df_col.index[-1], periods=13, freq='M')[1:]
        forecast = pd.Series(forecast, index=forecast_index)
        forecast_list.append(forecast)

    forecast1, forecast2, forecast3, forecast4, forecast5, forecast6, forecast7, forecast8, forecast9, forecast10 = forecast_list
    pred1 = forecast1['2023-12-31']
    pred2 = forecast2['2023-12-31']
    pred3 = forecast3['2023-12-31']
    pred4 = forecast4['2023-12-31']
    pred5 = forecast5['2023-12-31']
    pred6 = forecast6['2023-12-31']
    pred7 = forecast7['2023-12-31']
    pred8 = forecast8['2023-12-31']
    pred9 = forecast9['2023-12-31']
    pred10 = forecast10['2023-12-31']
    pred_total = pred1 + pred2 + pred3 + pred4 + pred5 + pred6 + pred7 + pred8 + pred9 + pred10


    # 파이플롯
    ratio = [pred1/pred_total, pred2/pred_total, pred3/pred_total, pred4/pred_total, pred5/pred_total, pred6/pred_total, pred7/pred_total, pred8/pred_total, pred9/pred_total, pred10/pred_total]
    labels = ['커뮤니케이션이 적은 집단',
           '평일 외출이 적은 집단', '휴일 외출이 적은 집단', '출근소요시간 및 근무시간이 많은 집단',
           '외출이 매우 적은 집단(전체)', '외출이 매우 많은 집단', '동영상서비스 이용이 많은 집단',
           '생활서비스 이용이 많은 집단', '재정상태에 대한 관심집단', '외출-커뮤니케이션이 모두 적은 집단(전체)']

    fig = px.pie(values=ratio, names=labels, hole=0.3)
    fig.update_traces(textposition='inside', textinfo='percent+label')
    fig.update_layout(font=dict(size=16))
    
    return fig

########################### 페이지 코드 작성 ###########################################
st.set_page_config(layout="wide")

st.markdown("## 사회복지사 통계 포털")
tab1, tab2, tab3 = st.tabs(["IoT 통계", "감정분석 통계", "1인가구 집단"])

with tab1: # IoT 통계
    
    st.markdown('통계')

    t1_col1_1, t1_col1_2 = st.columns([0.5, 0.5])
    with t1_col1_1:
        st.markdown('기가지니 감정 분류 통계')
    with t1_col1_2:
        st.markdown('10개 집단 시계열 차트')
    
    st.markdown('결과값')


with tab2: # 감정분석 통계
    st.markdown('통계')
    t2_col1_1, t2_col1_2 = st.columns([0.5, 0.5])


with tab3: # 1인가구 집단 통계

    t3_col1_1, t3_col1_2, t3_col1_3, t3_col1_4 = st.columns([0.2,0.2,0.2,0.2]) # 조회할 값 선택
    with t3_col1_1:
        st.subheader('**관심 집단 선택**')
        group = st.selectbox(" ", ['커뮤니케이션이 적은 집단','평일 외출이 적은 집단','휴일 외출이 적은 집단','출근소요시간 및 근무시간이 많은 집단','외출이 매우 적은 집단(전체)','외출이 매우 많은 집단','동영상서비스 이용이 많은 집단','생활서비스 이용이 많은 집단','재정상태에 대한 관심집단','외출-커뮤니케이션이 모두 적은 집단(전체)'])
    with t3_col1_2:
        st.subheader('**성별 선택**')
        gender = st.selectbox(' ', ['남성', '여성'])
    with t3_col1_3:
        st.subheader('**연령대 선택**')
        age = st.selectbox(' ', ['20대', '30대', '40대', '50대', '60대', '70대'])
    with t3_col1_4:
        st.subheader('**자치구 선택**')
        region = st.selectbox(' ', ['강남구', '강동구', '강북구', '강서구', '관악구', '광진구',  '구로구', '금천구', '노원구', '도봉구', '동대문구', '동작구', '마포구', '서대문구', '서초구', '성동구', '성북구', '송파구', '양천구', '영등포구', '용산구', '은평구', '종로구', '중구', '중랑구'])
    
    st.header(' ')
    
    # if st.button('선택'):
    #     st.write(region, age, gender, group)
    # else:
    #     st.write('조회할 값을 선택하세요.')
    

    # 버튼 스타일 변경
    button_style = """
        <style>
        .stButton button {
            background-color: #5EBAB8;
            color: white;
            padding: 0.5em 1em;
            border-radius: 0.3em;
            border: none;
        }
        </style>
    """

    # 버튼 생성
    st.markdown(button_style, unsafe_allow_html=True)
    if st.button('선택'):
        st.write(region, age, gender, group)
    else:
        st.write('버튼을 클릭하세요.')
    
    st.header(' ')
    st.header(' ')

        
    t3_col2_1, t3_col2_2 = st.columns([0.8, 0.2]) # 시계열차트
    with t3_col2_1:
        st.subheader(f'**{group}에 속하는 1인가구의 수**')
        Lone_Person_Dataset_Loader(group, region, gender, age)
    with t3_col2_2:
        st.subheader('**예측값**')
        st.header(' ')
        pred(group, region, gender, age)

    st.header(' ')
    st.header(' ')

    
    t3_col3_1, t3_col3_2 = st.columns([0.5, 0.5]) # 파이차트
    with t3_col3_1:
        st.subheader(f'**현재 {region} {age} {gender} 1인가구 집단 비중**')
        fig = piechart(region, gender, age)
        st.plotly_chart(fig)  

    with t3_col3_2:
        st.subheader(f'**미래 {region} {age} {gender} 1인가구 집단 비중**')
        fig2 = piechart_pred(region, gender, age)
        st.plotly_chart(fig2) 

Overwriting bigproject_testpage.py


In [5]:
!streamlit run bigproject_testpage.py

^C
